In [2]:
# Import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey, Boolean
from sqlalchemy.orm import sessionmaker, relationship

In [3]:
# Create engine
engine = create_engine("sqlite:///birds.sqlite")

In [4]:
# Declaring a base object to model the table as an ORM class
Base = declarative_base()

In [5]:
# Define ORM class for `regions` table
class Regions(Base):
    
    __tablename__ = 'regions'

    id = Column(Integer, primary_key=True)
    state_abbr = Column(Text)
    region = Column(Text)
    region2 = Column(Text)
    state = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.state_abbr}"

In [6]:
# Define ORM class for `speciesList` table
class Species(Base):
    
    __tablename__ = 'species'

    id = Column(Integer, primary_key=True)
    Scientific_Name = Column(Text)
    Common_Name = Column(Text)
    Species_Code = Column(Text)
    Order = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.ScientificName}"

In [7]:
# !rm birds.sqlite

In [8]:
# Creating the table with `create_all`
Base.metadata.create_all(engine)

In [9]:
# Verifying table names in the database
engine.table_names()

['regions', 'species']

In [10]:
# Bulk insert each file into their respective table
def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile, encoding = "utf8")
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)
    
# Call the function to insert the data for each table
populate_table(engine, Regions.__table__, 'data/regions.csv')
populate_table(engine, Species.__table__, 'data/clean_speciesList.csv')

In [11]:
# Basic query to ensure data was entered correctly into the table `regions`
engine.execute("SELECT * FROM regions LIMIT 5").fetchall()

[(1, 'AK', 'Alaska', 'Alaska', 'Alaska'),
 (2, 'KS', 'Midwest', 'Central Plains', 'Kansas'),
 (3, 'NE', 'Midwest', 'Central Plains', 'Nebraska'),
 (4, 'OK', 'Midwest', 'Central Plains', 'Oklahoma'),
 (5, 'MI', 'Midwest', 'Great Lakes', 'Michigan')]

In [12]:
# Basic query to ensure data was entered correctly into the table `species`
engine.execute("SELECT * FROM species LIMIT 5").fetchall()

[(1, 'Struthio camelus', 'Common Ostrich', 'ostric2', 'Struthioniformes'),
 (2, 'Struthio molybdophanes', 'Somali Ostrich', 'ostric3', 'Struthioniformes'),
 (3, 'Struthio camelus/molybdophanes', 'Common/Somali Ostrich', 'y00934', 'Struthioniformes'),
 (4, 'Rhea americana', 'Greater Rhea', 'grerhe1', 'Rheiformes'),
 (5, 'Rhea pennata', 'Lesser Rhea', 'lesrhe2', 'Rheiformes')]